In [1]:
import pandas as pd
import re
import string
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

%matplotlib inline

In [2]:
train_df = pd.read_json("../../kaggleData/2sigma/train.json")
test_df = pd.read_json("../../kaggleData/2sigma/test.json")
print train_df.shape
print test_df.shape

(49352, 15)
(74659, 14)


In [3]:
#simple preprocessing
train_df["features"] = train_df["features"].apply(lambda x:["_".join(i.split(" ")).lower().strip().replace('-','_') \
                                                            for i in x])
test_df["features"] = test_df["features"].apply(lambda x:["_".join(i.split(" ")).lower().strip().replace('-','_')\
                                                          for i in x])

In [4]:
def featureList(train_df,test_df,limit = 0.001):
    #acquiring the feature lists
    features_in_train = train_df["features"].apply(pd.Series).unstack().reset_index(drop = True).dropna().value_counts()
    features_in_train.sort(ascending  = False)
    features_in_test = test_df["features"].apply(pd.Series).unstack().reset_index(drop = True).dropna().value_counts()
    features_in_test.sort(ascending  = False)
    
    filtered_features_in_train = features_in_train[features_in_train > limit*len(train_df)]
    filtered_features_in_test = features_in_test[features_in_test > limit*len(test_df)]
    accept_list = set(filtered_features_in_train.index).union(set(filtered_features_in_test.index))
    return accept_list

def featureMapping(train_df,test_df,feature_list):
    for feature in feature_list:
        #add the feature column for both
        #if feature in the row, then set the value for (row,feature) to 1
        train_df['with_'+feature]=train_df['features'].apply(lambda x : 1 if feature in x else 0)
        test_df['with_'+feature]=test_df['features'].apply(lambda x : 1 if feature in x else 0)
    return

In [5]:
preaccept_list = list(featureList(train_df,test_df,limit = 0.0001))
#featureMapping(train_df,test_df,accept_list)

C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting


In [6]:
def clean(s):
    x = s.replace("-", "")
    x = x.replace(" ", "")
    x = x.replace("twenty four hour", "24")
    x = x.replace("24/7", "24")
    x = x.replace("24hr", "24")
    x = x.replace("24-hour", "24")
    x = x.replace("24hour", "24")
    x = x.replace("24 hour", "24")
    
    x = x.replace("common", "cm")
    x = x.replace("concierge", "doorman")
    x = x.replace("bicycle", "bike")
    x = x.replace("private", "pv")
    x = x.replace("deco", "dc")
    x = x.replace("decorative", "dc")
    x = x.replace("onsite", "os")
    x = x.replace("outdoor", "od")
    x = x.replace("ss appliances", "stainless")
    
    x = x.replace("high_",'hi')
    x = x.replace("high",'hi')
    x = x.replace("hi_",'hi')
   
    x = x.replace("live","lv")
    x = x.replace("pre_","pre")
    x = x.replace("all","al")
    x = x.replace("flex","fl")
    x = x.replace("residents","rs")
    x = x.replace("close","cl")
    x = x.replace("duplex","dp")
    x = x.replace("share","sh")
    x = x.replace("newly","new")
    x = x.replace("on_site","os")
    
    x = x.replace("24_hour","24")
    x = x.replace("full_time","24")


    return x

def feature_hash(x):
    cleaned = clean(x, uniq)
    key = cleaned[:4].strip()
    return key

In [7]:
key2original = defaultdict(list)
origin2key =  {}
k = 4
for f in preaccept_list:
    cleaned = clean(f)
    key = cleaned[:k].strip()
    key2original[key].append(f)
    origin2key[f] = key

In [8]:
#handy modifying the origin2key[f] for some of the not satisfying values
train_df["features"] = train_df["features"].apply(lambda x : [origin2key[i] if i in origin2key else i for i in x ])
test_df["features"] = test_df["features"].apply(lambda x : [origin2key[i] if i in origin2key else i for i in x ])


In [13]:
accept_list = list(featureList(train_df,test_df,limit = 0.005))


C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting
C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: sort is deprecated, use sort_values(inplace=True) for INPLACE sorting


In [14]:
len(accept_list)

41

In [8]:
key2original

defaultdict(list,
            {u'1_mo': [u'1_month_free'],
             u'24_d': [u'24_hour_doorman',
              u'24hr_doorman',
              u'24/7_concierge',
              u'24/7_doorman_concierge',
              u'full_time_doorman',
              u'24/7_doorman'],
             u'a_fu': [u'a_full_service_luxury_highrise'],
             u'actu': [u'actual_photos!', u'actual_apt._photos'],
             u'air_': [u'air_conditioning'],
             u'al_p': [u'all_pets_ok'],
             u'al_u': [u'all_utilities_included'],
             u'assi': [u'assigned_parking_space'],
             u'atte': [u'attended_lobby'],
             u'back': [u'backyard'],
             u'balc': [u'balcony'],
             u'base': [u'basement_storage'],
             u'bask': [u'basketball_court'],
             u'bike': [u'bike_storage', u'bike_room'],
             u'bill': [u'billiards_table_and_wet_bar',
              u'billiards_room',
              u'billiards'],
             u'brea': [u'breakfast_